In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_openai langchain_community langchain_core tavily-python wikipedia langchain_google_vertexai
%pip install -U psycopg psycopg-pool langgraph langgraph-checkpoint-postgres
%pip install --upgrade pip
%pip install "psycopg[binary,pool]"
%pip install langchain-google-genai
%pip install -U google-generativeai
%pip install elevenlabs

## Imports

In [2]:
import os, getpass

# def _set_env(var: str):
#     os.environ[var] = getpass.getpass(f"{var}: ")
#     print(os.environ[var])
os.environ["OPENAI_API_KEY"] = (
    "sk-proj-eyRmwxaT2HsjbPNPDryOT3BlbkFJtPYuHzf9iqpPwbl3kQlE"
)
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
#     "/Users/arpitkjain/credentials/dhhs-bsi-2021152-dev-e32b82a1af44.json"
# )
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/arpitkjain/credentials/bridgescapture-deloitte-poc1-policyengine-worker.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] ="/Users/arpitkjain/credentials/chronicles-of-ai-9b78920156c5.json"
os.environ["CREDIT_BASE_URL"] = "https://staging.cliniq360.com"
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0)
# llm_flash = ChatOpenAI(model="gpt-4o-mini-2024-07-18", temperature=0)
from langchain_google_vertexai import ChatVertexAI
from langchain.document_loaders import PyPDFLoader

# llm = ChatVertexAI(
#     model="gemini-1.5-pro-001",
#     temperature=0.0,
#     project="dhhs-bsi-2021152-dev",
#     location="us-central1",
# )

# llm_flash = ChatVertexAI(
#     model="gemini-1.5-flash-001",
#     temperature=0.0,
#     project="dhhs-bsi-2021152-dev",
#     location="us-central1",
# )

# llm_flash = ChatVertexAI(
#     model="gemini-1.5-flash-001",
#     temperature=0.0,
#     project="bridgescapture-deloitte-poc1",
#     location="us-central1",
# )
# llm_pro = ChatVertexAI(
#     model="gemini-1.5-pro-001",
#     temperature=0.0,
#     project="bridgescapture-deloitte-poc1",
#     location="us-central1",
# )

llm_flash = ChatVertexAI(
    model="gemini-1.5-flash-002",
    temperature=0.0,
    project="chronicles-of-ai",
    location="us-central1",
)
llm_pro = ChatVertexAI(
    model="gemini-1.5-pro-002",
    temperature=0.0,
    project="chronicles-of-ai",
    location="us-central1",
)

In [10]:
class OpenAIPrompts:
    def __init__(self):
        self.collector_instructions = """
        You are a helpful assistant tasked with collecting customer information in order to complete a loan application form.
        The information you need to collect includes: {required_fields}
        Generate questions to be asked to the customer to collect the required information.
        Initial Check: Analyze the already collected information provided in {customer_info} and identify missing values.
        If no information is available, start by asking the first question.
        Think step-by-step and carefully verify each piece of information before moving to the next.Ask only for the missing information 
        and avoid repeating questions for details that have already been collected.
        Final Validation: After going through the list, confirm that all details have been collected.
            - If all required information is available, reply only: "ALL DATA COLLECTED"
            - If there are still missing pieces of information, generate a question targeting those fields only.
        Now, proceed step-by-step and analyze {customer_info}.
        """

        self.extractor_instructions = """You are tasked with extracting the user details from the customer's response.
        Question asked to the user: {agent_question}.
        User response: {user_answer}.
        If information is not present in user_answer, return 'NA' for the respective field."""

        self.intent_classification_instructions = """You are tasked to identify the intent from the user message.
        The user could either agree to the information or ask for updates. Classify the intent accordingly.
        User message: {user_answer}."""

        self.account_collector_instructions = """
       You are a helpful assistant tasked with collecting customer information in order to complete a loan application form.
        The information you need to collect includes: {required_fields}
        Here’s how you should proceed:
        1. Initial Check: Analyze the already collected information provided in {customer_info} and identify missing values.
        2. Chain of Thought: Think step-by-step and carefully verify each piece of information before moving to the next.
            - Ask only for the missing information and avoid repeating questions for details that have already been collected.
        3. Final Validation: After going through the list, confirm that all details have been collected.
            - If all required information is available, reply only: "ALL DATA COLLECTED"
            - If there are still missing pieces of information, generate a question targeting those fields only.
        Now, proceed step-by-step and analyze {customer_info}.
        """

        self.account_extractor_instructions = """You are tasked with extracting the user details from the customer's response. 
            customer_response: {user_message}
            If information is not present in customer_response, return 'NA' for the respective field."""

        self.account_extractor_instructions = """You are tasked with extracting the user details from the customer's response. 
            customer_response: {user_message}
            If information is not present in customer_response, return 'NA' for the respective field."""

        self.summarise_loan_agreement_instructions = """
        You will be given different passages from a loan agreement document one by one. 
        Act as a financial advisor and provide a summary of the following text. 
        Your result must be detailed and atleast 2 paragraphs. 
        When summarizing, directly dive into the descriptions from the text without using introductory phrases.
        Directly address the loan details, important terms and conditions, and repayment stategies, encapsulating the essence and 
        significant details from the text. Keep a conversational tone. Generate the summary in 25 words only.
        Passage:
        ```{text}```
        SUMMARY:
        """

        self.qna_instructions = """
        Given is the text from loan agreement document.
        Act as a financial advisor and try to answer the user query about the loan agreement strictly based on the text provided. 
        Your result should be precise, to the point and detailed.
        Keep a conversational tone while answering the query. 
        If the user query cannot be answered from the text provided, "Sorry, I cannot comment on that. I can only answer your queries regarding the loan agreement document.".
        Text:
        ```{loan_agreement_text}```
        User Query: {user_query}.
        Answer:
        """

        self.user_intent_1_instructions = """User message: {user_message}.
        Classify the user_message provided above between two categories: ["question", "acknowledgment"]. Respond in one word."""

        self.offer_summary_instructions = """Offer details: {offer_list}.
        Act as a financial adviser. From the credit offer list provided above, help customer understand each credit offer in simple paragraph focusing on important information. 
        Keep the tone conversational and maximum 3 lines per offer."""

        self.offer_qna_instructions = """Offer details : {offer_list}.
        Try to answer the user_query in brief based on the offer details. If applicable, provide the details from the offer details above. Keep the tone conversational.
        user_query: {user_query}"""


class GeminiPrompts:
    def __init__(self):

        # self.collector_instructions = """
        # You are a helpful assistant tasked with collecting customer information in order to complete a loan application form.
        # The information you need to collect includes: {required_fields}
        # Here’s how you should proceed:
        # 1. Initial Check: Analyze the already collected information provided in {customer_info} and identify missing values.
        # 2. Sequential Processing: For each required detail (in the list above), check if it has been provided.
        #     - If the detail is missing, ask the customer a follow-up question to gather the information for that specific field.
        #     - If the detail is present, move to the next field without prompting for it again.
        # 3. Handle End-of-Sequence Issues: Pay special attention to the last element in the list (Income, Address Line 2) and ensure that it is properly processed, even if it is at the end.
        #     - If Income or Address Line 2 is already provided, do not ask for it again. If it's missing, generate a question specifically for it.
        #     - Use explicit end-sequence handling to ensure that no details are missed due to an indexing or sequence error.
        # 4. Chain of Thought: Think step-by-step and carefully verify each piece of information before moving to the next.
        #     - Ask only for the missing information and avoid repeating questions for details that have already been collected.
        # 5. Final Validation: After going through the list, confirm that all details have been collected.
        #     - If all required information is available, return: "ALL DATA COLLECTED"
        #     - If there are still missing pieces of information, generate a question targeting those fields only.
        # Now, proceed step-by-step and analyze {customer_info}.
        # """
        self.collector_instructions = """
        You are a helpful assistant tasked with collecting customer information in order to complete a loan application form.
        The required_fields: {required_fields}. This is divided into the following sections: personal_information,contact_information, address_information, employment_information, and other_information.
        Here’s how you should proceed:
        1. Initial Check: Analyze the already collected information customer_info in {customer_info} and identify missing values.
        2. Sequential Processing: For each required field in required_fields , check if value is already collected for all the keys in the sections.
            - If the value is missing, ask the customer a follow-up question to gather the information for all the missing fields in a section together.
            - If the detail is present, move to the next key without prompting for it again.
            - If all the values for a section are already collected, move to the next section.
        3. Chain of Thought: Think step-by-step and carefully verify each piece of information before moving to the next.
            - Ask only for the missing information and avoid repeating questions for details that have already been collected.
        4. Final Validation: After going through the list, confirm that all details have been collected.
            - If all required information is available, return: "ALL DATA COLLECTED"
            - If there are still missing pieces of information, generate a question targeting those fields only.
        Now, proceed step-by-step and analyze {customer_info}.
        """

        self.collector_instructions_hi = """
        You are a helpful assistant tasked with collecting customer information in order to complete a loan application form.
        The required_fields: {required_fields}. This is divided into the following sections: personal_information,contact_information, address_information, employment_information, and other_information.
        Here’s how you should proceed:
        1. Initial Check: Analyze the already collected information customer_info in {customer_info} and identify missing values.
        2. Sequential Processing: For each required field in required_fields , check if value is already collected for all the keys in the sections.
            - If the value is missing, ask the customer a follow-up question to gather the information for all the missing fields in a section together.
            - If the detail is present, move to the next key without prompting for it again.
            - If all the values for a section are already collected, move to the next section.
        3. Chain of Thought: Think step-by-step and carefully verify each piece of information before moving to the next.
            - Ask only for the missing information and avoid repeating questions for details that have already been collected.
        4. Final Validation: After going through the list, confirm that all details have been collected.
            - If all required information is available, return: "ALL DATA COLLECTED"
            - If there are still missing pieces of information, generate a question in Hindi targeting those fields only.
        Now, proceed step-by-step and analyze {customer_info}.
        """

        self.extractor_instructions = """You are tasked with extracting the user details from the customer's response.
        Question asked to the user: {agent_question}.
        User response: {user_answer}.
        If the information is not present in the user response, return 'None'."""

        self.intent_classification_instructions = """You are tasked to identify the intent from the user message.
        The user could either agree to the information or ask for updates. Classify the intent accordingly.
        User message: {user_answer}."""

        self.account_collector_instructions = """
        You are a helpful assistant tasked with collecting customer information in order to complete a loan application form.
        The information you need to collect includes: {required_fields}
        Here’s how you should proceed:
        1. Initial Check: Analyze the already collected information provided in {customer_info} and identify missing values.
        2. Sequential Processing: For each required detail (in the list above), check if it has been provided.
            - If the detail is missing, ask the customer a follow-up question to gather the information for that specific field.
            - If the detail is present, move to the next field without prompting for it again.
        3. Handle End-of-Sequence Issues: Pay special attention to the last element in the list (ifscCode) and ensure that it is properly processed, even if it is at the end.
            - If ifscCode is already provided, do not ask for it again. If it's missing, generate a question specifically for it.
            - Use explicit end-sequence handling to ensure that no details are missed due to an indexing or sequence error.
        4. Chain of Thought: Think step-by-step and carefully verify each piece of information before moving to the next.
            - Ask only for the missing information and avoid repeating questions for details that have already been collected.
            - Ask question in group of 4 required_fields.
        5. Final Validation: After going through the list, confirm that all details have been collected.
            - If all required information is available, return: "ALL DATA COLLECTED"
            - If there are still missing pieces of information, generate a question targeting those fields only.
        Now, proceed step-by-step and analyze {customer_info}.
        Keep a conversational tone.
        """

        self.account_extractor_instructions = """You are tasked with extracting the user details from the customer's response. 
            If values are not present, return None.
            customer response: {user_message}"""

        self.summarise_loan_agreement_instructions = """
        You will be given different passages from a loan agreement document one by one. 
        Act as a financial advisor and provide a summary of the following text. 
        Your result must be detailed but brief. Only focus on the key points and important details.
        When summarizing, directly dive into the descriptions from the text without using introductory phrases.
        Directly address the loan details, important terms and conditions, and repayment stategies, encapsulating the essence and 
        significant details from the text. Keep a conversational tone.
        Passage:
        ```{text}```
        SUMMARY:
        """

        self.summarise_loan_agreement_instructions_hi = """
        You will be given different passages from a loan agreement document one by one. 
        Act as a financial advisor and provide a summary of the following text in Hindi. 
        Your result must be detailed but brief. Only focus on the key points and important details.
        When summarizing, directly dive into the descriptions from the text without using introductory phrases.
        Directly address the loan details, important terms and conditions, and repayment stategies, encapsulating the essence and 
        significant details from the text. Keep a conversational tone.
        Passage:
        ```{text}```
        SUMMARY:
        """

        self.qna_instructions = """
        Given is the text from loan agreement document.
        Act as a financial advisor and try to answer the user query about the loan agreement strictly based on the text provided. 
        Your result should be precise, to the point and detailed.
        Keep a conversational tone while answering the query. 
        If the user query cannot be answered from the text provided, "Sorry, I cannot comment on that. I can only answer your queries regarding the loan agreement document.".
        Text:
        ```{loan_agreement_text}```
        User Query: {user_query}.
        Answer:
        """

        self.qna_instructions_hi = """
        Given is the text from loan agreement document.
        Act as a financial advisor and try to answer the user query about the loan agreement strictly based on the text provided. 
        Answer should be generated in Hindi.
        Your result should be precise, to the point and detailed.
        Output should be in maximum 200 characters including spaces and special characters. Keep a conversational tone while answering the query. 
        If the user query cannot be answered from the text provided, "Sorry, I cannot comment on that. I can only answer your queries regarding the loan agreement document.".
        Text:
        ```{loan_agreement_text}```
        User Query: {user_query}.
        Answer:
        """

        self.user_intent_1_instructions = """User message: {user_message}.
        Classify the user_message provided above between two categories: ["question", "acknowledgment"]. Respond in one word."""

        self.user_intent_2_instructions = """You are a helpful assistant that understands user intent.
        A user has provided a message related to an offer they were shown. Output should strictly be one of the following:
        1. "proceed_with_current_offer" - If the user likes the offer and wants to proceed with it.
        2. "get_more_offers" - If the user wants to explore other offers.
        3. "get_more_details" - If the user has doubts or questions about the offer and needs clarification.
        Analyze the following sentence and provide your response with one of the options above.
        user_message: {user_message}.
        Respond in one word."""

        self.offer_summary_instructions = """Offer details: {offer_list}.
        Act as a financial adviser. From the credit offer list provided above, help customer understand each credit offer in simple paragraph focusing on important information. 
        Output should be in maximum 200 english characters including spaces and special characters. Keep the tone conversational."""

        self.offer_summary_instructions_hi = """Offer details: {offer_list}.
        Act as a financial adviser. From the credit offer list provided above, help customer understand each credit offer in simple paragraph focusing on important information. 
        Keep the tone conversational and generate output in Hindi. Avoid any salutations or greetings.
        Output should be in maximum 200 characters including spaces and special characters."""

        self.offer_qna_instructions = """Offer details : {offer_list}.
        Try to answer the user_query in brief based on the offer details. If applicable, provide the details from the offer details above. Keep the tone conversational.
        user_query: {user_query}"""

        self.offer_qna_instructions_hi = """Offer details : {offer_list}.
        Try to answer the user_query in brief based on the offer details. Keep the tone conversational and generate output in Hindi. Avoid any salutations or greetings. If applicable, provide the details from the offer details above.
        Output should be in maximum 200 characters including spaces and special characters.
        user_query: {user_query}"""

In [11]:
offer_list = [
        {
            "offer_details": {
                "offer_item_id": "1d4b8433-8c2b-4e23-a53d-0db2cdca070e",
                "offer_current_action": "ON_INIT_ACC_DTL",
                "offer_row_id": 11388,
                "order_id": None,
                "loan_status": "INITIATED",
                "item_price": "135704.00",
                "INTEREST_RATE": "19.75%",
                "TERM": "36 months",
                "INTEREST_RATE_TYPE": "FIXED",
                "APPLICATION_FEE": "2% + GST",
                "FORECLOSURE_FEE": "4% + GST",
                "INTEREST_RATE_CONVERSION_CHARGE": "0",
                "DELAY_PENALTY_FEE": "36%",
                "OTHER_PENALTY_FEE": "1% + GST",
                "ANNUAL_PERCENTAGE_RATE": "21.5",
                "REPAYMENT_FREQUENCY": "MONTHLY",
                "NUMBER_OF_INSTALLMENTS_OF_REPAYMENT": "36",
                "TNC_LINK": "https://finance.adityabirlacapital.com/privacy-policy#:~:text=ABFL%20treats%20your%20personal%20information,and%20regulatory%20processes%2C%20or%20to",
                "COOL_OFF_PERIOD": "P3D",
                "INSTALLMENT_AMOUNT": "14815.00 INR",
                "MIN_INTEREST_RATE": "14%",
                "MAX_INTEREST_RATE": "32%",
                "MIN_TENURE": "12 months",
                "MAX_TENURE": "24 months",
                "MIN_LOAN_AMOUNT": "50000",
                "MAX_LOAN_AMOUNT": "500000"
            },
            "provider_details": {
                "name": "Aditya Birla Finance Limited",
                "short_desc": "Aditya Birla Finance Limited",
                "long_desc": "Aditya Birla Finance Limited",
                "images": [
                    {
                        "url": "https://abcscprod.azureedge.net/-/media/Feature/ABCL/ABFL/Sticky-Tab-Content/ABFLLogov11.webp?extension=webp&revision=6d918649-4a8f-4754-8e00-cf7601b99ef8&modified=20221111111234&h=387&w=431&la=en&hash=5B3653A133488E98B636182FCB8A1908",
                        "size_type": "sm"
                    }
                ],
                "GRO_NAME": "Mr. Arjit Sen",
                "GRO_EMAIL": "grievance.finance@adityabirlacapital.com",
                "GRO_CONTACT_NUMBER": "080 – 45860196",
                "GRO_DESIGNATION": "Nodal Grievance Redressal Officer",
                "GRO_ADDRESS": "Aditya Birla Finance Limited, Tower 1, 18th Floor Jupiter mill compound 841, Senapati Bapat Marg, Elphinstone Road, Mumbai 400013",
                "CUSTOMER_SUPPORT_LINK": "https://finance.adityabirlacapital.com/customer-services",
                "CUSTOMER_SUPPORT_CONTACT_NUMBER": "080 – 45860196",
                "CUSTOMER_SUPPORT_EMAIL": "grievance.finance@adityabirlacapital.com",
                "LSP_NAME": "NA",
                "LSP_EMAIL": "NA",
                "LSP_CONTACT_NUMBER": "NA",
                "LSP_ADDRESS": "NA"
            },
            "quote_details": {
                "PRINCIPAL": "100000",
                "INTEREST": "33344.0",
                "PROCESSING_FEE": "2360.0",
                "OTHER_UPFRONT_CHARGES": "0.00",
                "INSURANCE_CHARGES": "0.00",
                "NET_DISBURSED_AMOUNT": "97640.0",
                "OTHER_CHARGES": "0.00",
                "quote_price": "135704"
            },
            "payment_details": []
        },
        {
            "offer_details": {
                "offer_item_id": "3b55859a-0d94-427e-b1ef-9deab8cb6928",
                "offer_current_action": "ON_SELECT_LOAN_AMT",
                "offer_row_id": 11386,
                "order_id": None,
                "loan_status": "DISCOVERY",
                "item_price": "442300",
                "INTEREST_RATE": "12%",
                "TERM": "5 months",
                "INTEREST_RATE_TYPE": "FIXED",
                "APPLICATION_FEE": "0",
                "FORECLOSURE_FEE": "0.5%",
                "INTEREST_RATE_CONVERSION_CHARGE": "0",
                "DELAY_PENALTY_FEE": "5%",
                "OTHER_PENALTY_FEE": "1%",
                "ANNUAL_PERCENTAGE_RATE": "5%",
                "REPAYMENT_FREQUENCY": "MONTHLY",
                "NUMBER_OF_INSTALLMENTS_OF_REPAYMENT": "5",
                "TNC_LINK": "https://pramaan.ondc.org/beta/preprod/mock/seller/document/tnc.pdf",
                "COOL_OFF_PERIOD": "PT5M",
                "INSTALLMENT_AMOUNT": "88460.00",
                "CONSENT_HANDLER": None,
                "MIN_INTEREST_RATE": "9%",
                "MAX_INTEREST_RATE": "15%",
                "MIN_TENURE": "5 months",
                "MAX_TENURE": "5 years",
                "MIN_LOAN_AMOUNT": "50000",
                "MAX_LOAN_AMOUNT": "5000000"
            },
            "provider_details": {
                "images": [
                    {
                        "size_type": "sm",
                        "url": "https://ondc.org/assets/theme/images/ondc_registered_logo.svg?v=399788fda7"
                    }
                ],
                "long_desc": "ONDC Bank Ltd, India.",
                "name": "ONDC Bank",
                "short_desc": "Ondc Bank Ltd",
                "GRO_NAME": "ONDC",
                "GRO_EMAIL": "lifeline@ondc.com",
                "GRO_CONTACT_NUMBER": "1860 266 7766",
                "GRO_DESIGNATION": "Nodal Grievance Redressal Officer",
                "GRO_ADDRESS": "One Indiabulls centre, Tower 1, 18th Floor Jupiter mill compound 841, Senapati Bapat Marg, Elphinstone Road, Mumbai 400013",
                "CUSTOMER_SUPPORT_LINK": "https://buy.ondc.org/buy/GrievanceRedStep.htm?execution=e1s1",
                "CUSTOMER_SUPPORT_CONTACT_NUMBER": "1800 1080",
                "CUSTOMER_SUPPORT_EMAIL": "customer.care@ondc.com",
                "LSP_NAME": "ONDC_BANK_LSP",
                "LSP_EMAIL": "lsp@ondcbank.com",
                "LSP_CONTACT_NUMBER": "1860 266 7766",
                "LSP_ADDRESS": "One Indiabulls centre, Tower 1, 18th Floor Jupiter mill compound 841, Senapati Bapat Marg, Elphinstone Road, Mumbai 400013"
            },
            "quote_details": {
                "PRINCIPAL": "400000",
                "INTEREST": "40000",
                "PROCESSING_FEE": "1800",
                "OTHER_UPFRONT_CHARGES": "0",
                "INSURANCE_CHARGES": "500",
                "NET_DISBURSED_AMOUNT": "397700",
                "OTHER_CHARGES": "0",
                "quote_price": "442300"
            },
            "payment_details": []
        }
    ],
offer_summary_instructions = GeminiPrompts().offer_summary_instructions_hi
offer_summary_prompt = offer_summary_instructions.format(offer_list=offer_list)
offer_summary = llm_flash.invoke(offer_summary_prompt)
prompt = f"Consider all the numeric values in the text and convert them into words. Currency is in Indian Rupees. input_text: {offer_summary.content}"
summary_in_words = llm_flash.invoke(prompt)
print(summary_in_words.content)


offer_qna_instructions = GeminiPrompts().offer_qna_instructions_hi
offer_qna_prompt = offer_qna_instructions.format(
    offer_list=offer_list, user_query="Which one is better out of these two?"
)
llm_response = llm_flash.invoke(offer_qna_prompt)
prompt = f"Consider all the numeric values in the text and convert them into words. Currency is in Indian Rupees. Keep the tone conversational. Generate output in Hindi. input_text: {llm_response.content}"
llm_response_in_words = llm_flash.invoke(prompt)
print(llm_response_in_words.content)

पहला ऑफर: एक लाख रुपये लोन, उन्नीस दशमलव पचहत्तर प्रतिशत ब्याज, छत्तीस महीने की अवधि, मासिक किश्त चौदह हजार आठ सौ पंद्रह रुपये। दूसरा ऑफर: चार लाख रुपये लोन, बारह प्रतिशत ब्याज, पाँच महीने की अवधि, मासिक किश्त अट्ठावन हजार चार सौ साठ रुपये। दूसरे ऑफर में कम अवधि और कम ब्याज है। अपनी ज़रूरत के हिसाब से चुनें।

अच्छा, दोनों में से कौन सा बेहतर है, ये आपकी ज़रूरत और बजट पर निर्भर करता है। पहला लोन आदित्य बिड़ला फाइनेंस लिमिटेड से है, जिसमें ब्याज दर ज़्यादा है, लेकिन  लोन की अवधि लंबी है। दूसरा ओएनडीसी बैंक से है, जिसमें ब्याज दर कम है और अवधि भी कम है।  मुझे  ब्याज दर और अवधि के बारे में  संख्यात्मक जानकारी नहीं मिली है, इसलिए मैं  उनको शब्दों में नहीं बदल पा रहा हूँ।



In [26]:
from pydantic import BaseModel, Field
import re
class UserIntent2(BaseModel):
    user_intent: str = Field(
        description="Classification of user message  in get_more_details or get_more_offers or proceed_with_current_offer"
    )


structured_llm = llm_flash.with_structured_output(UserIntent2)
# user_intent_instructions = GeminiPrompts().user_intent_2_instructions
# prompt = """You are a helpful assistant that understands user intent. A user has provided a message related to an offer they were shown. Output should strictly be one of the following:
# 1. "proceed_with_current_offer" - If the user likes the offer and wants to proceed with it.
# 2. "get_more_offers" - If the user wishes to explore other offers.
# 3. "get_more_details" - If the user wishes to know more details about the offers presented.
# Analyze the following sentence and provide your response with one of the options above. user_message: Which offer is better?. Respond in one word."""
user_message="What is the interest rate"
prompt = f"""You are a helpful assistant that understands user sentiment and intent.
        A user has provided a sentence related to an offer they were shown.
        Your task is to determine if the user:
        1. Likes the offer and whant to proceed with it. In this case, respond with "proceed_with_current_offer".
        2. Has doubts or questions about the offer and needs clarification. In this case, respond with "get_more_details".
        3. Wants to explore other offers. In this case, respond with "get_more_offers".
        Analyze the following sentence and provide your response with one of the options above.
        user_message: {user_message}.
        Respond in one word."""

llm_response = structured_llm.invoke(prompt)
user_intent = llm_response.user_intent
user_intent = user_intent.lower().strip()
user_intent = re.sub("[^A-Za-z_]+", "", user_intent)
print(f"{user_intent=}")

user_intent='get_more_details'
